# Structure of PAV core counterexamples for $k = 8$: Dual certificates

This notebook contains code for verifying the claims in the proof of Lemma 4.4 (about committee size $k = 8$) regarding the optimum values of various linear programs. For this, it reads in pre-computed dual solutions and verifies their correctness, which involves simple exact (fractional) arithmetic.

In [1]:
import itertools
from functools import cache
import pickle
from fractions import Fraction
from utility_functions import powerset, show, utility, generate_wlog_partition, is_wlog

In [2]:
@cache
def harmonic_number(r):
    return sum(Fraction(1,i) for i in range(1,r+1))

@cache
def swapped(committee, x, y):
    return tuple(set(committee) - {x} | {y})

As explained in the paper, we may label the PAV committee w.l.o.g. as $0, 1, \ldots, k-1$. For $k = 8$, the only deviation that can occur is $T = (0, 1, 8, 9)$ (with appropriate labelling), and we may restrict the candidate set to $W \cup T = \{0, 1, \ldots, 9\}$.

In [3]:
num_alts = 10
k = 8
A = list(range(num_alts))
ballots = sorted(powerset(A))
committees = list(itertools.combinations(A, k))
pav_committee = tuple(A[:k])
swaps = [(x,y) for x in pav_committee for y in A if y not in pav_committee]
T = (0, 1, 8, 9)

To begin, we load the dual solutions from the file `dual_values_k8.pkl`. They were computed using the code in the notebook `primals-counterexample-k8-structure.ipynb`. After loading, `dual_solutions` is a dictionary whose keys are names of the different linear programs, and whose values are dictionaries that map names of dual variables to their values.

In [4]:
pkl_filename = "dual_values_k8.pkl"
dual_solutions = pickle.load(open(pkl_filename, "rb"))

## Structure of the linear programs

Each primal linear program discussed in the proof of Lemma 4.4 has the following structure, for some choice of objective coefficients $(c_A)_{A \in \mathcal{A}}$:

$$
\begin{aligned}
\max  \quad & \sum_{A \in \mathcal{A}} c_A p_A \\
\text{s.t.} \quad & \sum_{A\in \mathcal{A}} p_A && = 1,\\
& \sum_{A\in \mathcal{A}}p_A\,\Delta_{A,x,y} && \le 0 
   \quad\text{for each } x\in W, y\notin W,\\
& \sum_{A : u_A(T) > u_A(W)} p_A &&\ge  \frac{|T|}{k},\\
& p_A && \ge 0  \quad\text{for all }A.
\end{aligned}
$$

The dual linear program has the following structure:

$$
\begin{aligned}
\min \quad & \alpha - \frac{|T|}{k} \gamma \\
\text{s.t.} \quad & 
\alpha  + \sum_{x\in W, y\notin W} \beta_{x,y} \Delta_{A,x,y}
    - \gamma &&\ge c_A \quad\text{for all $A \in \mathcal{A}$ with $u_A(T) > u_A(W)$},\\
& \alpha + \sum_{x\in W, y\notin W} \beta_{x,y} \Delta_{A,x,y} &&\ge c_A \quad\text{for all $A \in \mathcal{A}$ with $u_A(T) \le u_A(W)$},\\
& \beta_{x,y} \ge 0,\quad
  \gamma \ge 0,\quad
  \alpha \in \mathbb{R}.
\end{aligned}
$$

The function `verify_dual` takes as input a dual solution (a dictionary mapping dual variable names to their values) and the objective coefficients $c_A$ for the primal linear program. It verifies that the dual solution is feasible and computes the corresponding dual objective value. If this equals the primal objective value claimed in the paper, this verifies the correctness of that claim.

In [5]:
def verify_dual_solution(dual_solution, objective):
    alpha = dual_solution["alpha"]
    beta = {(x, y) : dual_solution[f"beta_{x}_{y}"] for x, y in swaps}
    gamma = dual_solution["gamma"]
    assert all(beta[x, y] >= 0 for x in pav_committee for y in A if y not in pav_committee)
    assert gamma >= 0
    for ballot in ballots:
        lhs = alpha
        for x, y in swaps:
            lhs += beta[x, y] * (harmonic_number(utility(ballot, swapped(pav_committee, x, y))) - harmonic_number(utility(ballot, pav_committee)))
        if utility(ballot, T) > utility(ballot, pav_committee):
            lhs -= gamma
        if not lhs >= objective[ballot]:
            print(f"lhs = {lhs}, rhs = {objective[ballot]}")
        assert lhs >= objective[ballot]
    return alpha - Fraction(len(T), k) * gamma

## First program type: ballots in the blocking coalition

The first batch of programs concerns the frequency with which ballots supporting $T$ appear in the profile, with both $\{0, 1, 8\}$ and $\{0, 1, 9\}$ appearing with frequency $1/4$ (after restricting the candidate set to $W \cup T$).

Note that the dual solution will be $1/4$ for maximization problems and $-1/4$ for minimization problems, due to the way we formulated the dual linear program.

In [6]:
special_ballot = (0, 1, 8)
objective = {ballot: int(ballot == special_ballot) for ballot in ballots}
verify_dual_solution(dual_solutions["max_freq_018"], objective)

Fraction(1, 4)

In [7]:
special_ballot = (0, 1, 8)
objective = {ballot: -int(ballot == special_ballot) for ballot in ballots}
verify_dual_solution(dual_solutions["min_freq_018"], objective)

Fraction(-1, 4)

In [8]:
special_ballot = (0, 1, 9)
objective = {ballot: int(ballot == special_ballot) for ballot in ballots}
verify_dual_solution(dual_solutions["max_freq_019"], objective)

Fraction(1, 4)

In [9]:
special_ballot = (0, 1, 9)
objective = {ballot: -int(ballot == special_ballot) for ballot in ballots}
verify_dual_solution(dual_solutions["min_freq_019"], objective)

Fraction(-1, 4)

## Second program type: ballots outside the blocking coalition

The second batch of programs verifies that, besides $\{0, 1, 8\}$ and $\{0, 1, 9\}$, all other ballots are disjoint from $T$. We do this by considering each ballot that intersects $T$ and verifying that its maximum frequency in the profile is $0$.

The code below verifies this only for the ballots that are "canonical" to speed up the computation.

In [10]:
ballots_intersecting_T = [ballot for ballot in ballots if any(x in T for x in ballot)]
# restrict to canonical ballots
partition = generate_wlog_partition(A, [T, pav_committee])
ballots_intersecting_T = [ballot for ballot in ballots_intersecting_T if is_wlog(ballot, partition)]
for special_ballot in ballots_intersecting_T:
    if special_ballot == (0, 1, 8):
        # this ballot is analyzed in the first program type
        continue
    objective = {ballot: int(ballot == special_ballot) for ballot in ballots}
    assert verify_dual_solution(dual_solutions[f"max_freq_{show(special_ballot)}"], objective) == 0
print("All tests passed, dual solutions have objective value 0 for all ballots intersecting T")

All tests passed, dual solutions have objective value 0 for all ballots intersecting T


## Third program type: change of PAV score when removing a candidate

The third program computes by how much the PAV score of the committee $W$ changes if any one member of $W \setminus \{0,1\}$ is removed. Without loss of generality, we only need to consider removing $x = 2$. The answer is $-1/12$.

As before, note that the dual solution will be $-1/12$ for the maximization problem and $1/12$ for the minimization problem, due to the way we formulated the dual linear program.

In [11]:
x = 2 # candidate to delete
pav_committee_without_x = tuple(a for a in pav_committee if a != x)
objective = {ballot : harmonic_number(utility(ballot, pav_committee_without_x)) - harmonic_number(utility(ballot, pav_committee)) for ballot in ballots}
verify_dual_solution(dual_solutions[f"max_marginal_{x}"], objective)

Fraction(-1, 12)

In [12]:
x = 2 # candidate to delete
pav_committee_without_x = tuple(a for a in pav_committee if a != x)
objective = {ballot : -(harmonic_number(utility(ballot, pav_committee_without_x)) - harmonic_number(utility(ballot, pav_committee))) for ballot in ballots}
verify_dual_solution(dual_solutions[f"min_marginal_{x}"], objective)

Fraction(1, 12)